## Enderscope stage

This notebook shows how to connect and drive an enderscope stage from Python.

In [1]:
from enderscope import Stage, Panel
from serial_utils import serial_ports

In [2]:
# list available serial ports
serial_ports()

['/dev/tty.Bluetooth-Incoming-Port', '/dev/tty.usbserial-142130']

In [3]:
# connect to enderscope and create a stage object
s = Stage(serial_ports()[1], 115200)

In [10]:
# create a stage control panel
p = Panel(s)

GridspecLayout(children=(Button(description='North', layout=Layout(grid_area='widget001', height='auto', width…

Output()

In [11]:
# home the stage for positionning reproducibility
#s.home()
# move to a safe distance along the z axis, axis is x, y or z, distances are in millimeters
s.move_axis('z',10)

In [12]:
# read the stage position
myhome = s.get_position()

In [7]:
# position can be retrieved as a dict
s.get_position(dict=True)

{'X': -18.0, 'Y': -2.0, 'Z': 10.0, 'E': 0.0}

In [8]:
# move to (x,y(,z)) absolute coordinate
s.move_absolute(50,120)

In [9]:
# debug view the actual G-Code that was sent
s.move_absolute(20,30, debug=True)

G0 X 20 Y 30


In [10]:
# you can head towards standard directions, distances are always > 0
s.move_towards('east',70)

In [11]:
# you can move by some amount, relative to the current position
s.move_relative(0,0,5)

In [12]:
for i in range(15):
    s.move_relative(-1,0,0)

In [13]:
# a position is a tuple of length 2 or 3 to which you can move
pos = (20,20)
s.move_position(pos)

In [14]:
p = (40,40,10)
s.move_position(p)

In [17]:
# sending specific gcode to read device parameters
s.write_code(f"M503", debug=True)

echo:  G21    ; Units in mm (mm)
echo:  M149 C ; Units in Celsius

echo:; Filament settings: Disabled
echo:  M200 S0 D1.75
echo:; Steps per unit:
echo: M92 X80.00 Y80.00 Z400.00 E93.00
echo:; Maximum feedrates (units/s):
echo:  M203 X300.00 Y300.00 Z5.00 E25.00
echo:; Maximum Acceleration (units/s2):
echo:  M201 X3000.00 Y3000.00 Z100.00 E10000.00
echo:; Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>
echo:  M204 P3000.00 R3000.00 T3000.00
echo:; Advanced: B<min_segment_time_us> S<min_feedrate> T<min_travel_feedrate> X<max_x_jerk> Y<max_y_jerk> Z<max_z_jerk> E<max_e_jerk>
echo:  M205 B20000.00 S0.00 T0.00 X10.00 Y10.00 Z0.30 E5.00
echo:; Home offset:
echo:  M206 X0.00 Y0.00 Z0.00
echo:; Material heatup parameters:
echo:  M145 S0 H200.00 B60.00 F0
echo:  M145 S1 H240.00 B80.00 F0
echo:; PID settings:
echo:  M301 P19.98 I0.88 D50.29
echo:  M304 P462.10 I85.47 D624.59
echo:; Power-Loss Recovery:
echo:  M413 S1
echo:; UI Language:
echo:  M414 S1
M503


'ok\n'

In [18]:
s.write_code(f"M203 Z50") # increase z axis speed to 50
s.move_axis('z',40) # go up fast
s.write_code(f"M203 Z5") # restore z axis speed to 5
s.move_axis('z',-40) # go down slowly

In [19]:
# create an array of positions to perform a raster scan
import time
import numpy as np
origin = [50,120] # coordinates of first position
steps = [25,-25] # x and y offsets to next item
grid = np.array(list((x,y) for y in range(3) for x in range(4))) * steps + origin
for p in grid:
    s.move_position(p)
    s.finish_moves() 
    #time.sleep(0.5)
s.move_position(origin) # move back to origin


In [27]:
# do a x,y circle scan with oscillating z
time.sleep(10)
import math
radius = 30
center = (100,100)
curr_z = s.get_position()[2]
steps = 120
for a in range(steps):
    s.move_absolute(center[0]+radius*math.sin(2*math.pi*a/steps),
                    center[1]+radius*math.cos(2*math.pi*a/steps),
                    curr_z+5*math.sin(2*math.pi*(a*12)/steps))
